In [ ]:
! unzip ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip 
! unzip ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip 
! unzip ../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip 
! unzip ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip

In [ ]:
! pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import transformers
from tqdm.notebook import tqdm

In [ ]:
bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased") 
bert_model.trainable = False

In [ ]:
batch_size=32
max_len=128
EPOCHS=2

In [ ]:
data=pd.read_csv("./train.csv",nrows=75000)
test_labels=pd.read_csv("./test_labels.csv",nrows=75000)
# train_data.head() 
# print(data.loc[0])
# data.columns

# data["toxic"].value_counts()
data.head()
test_labels.head()

In [ ]:
row_no=list(data.shape)[0]
data["comment_text"]=data["comment_text"].map(lambda x:re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x))

In [ ]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'] 
sense_count_pd=pd.DataFrame(data[label_cols].value_counts()) 
sense_count_pd

In [ ]:
labels =  test_labels[label_cols].values

In [ ]:
# indexes = np.arange(12)
# np.array(labels[indexes], dtype="int32")
labels

In [ ]:
from sklearn.model_selection import train_test_split 
input_sen=data["comment_text"].values
# print(input_sen)
train_inputs,validation_inputs,train_labels,validation_labels=train_test_split(input_sen,labels,random_state=0,test_size=0.2) 


print(train_inputs.shape)
print(train_labels.shape)

print(validation_inputs.shape)
print(validation_labels.shape)
 

In [ ]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence): 
    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
         
        
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=128,
            return_attention_mask=True,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_tensors="tf",
        )   

        bert_output = bert_model(**encoded)
        
        sequence_output = bert_output.last_hidden_state 
         
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return sequence_output, labels
        else:
            return sequence_output

    def on_epoch_end(self): 
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [ ]:
train_dataset=BertSemanticDataGenerator(train_inputs,train_labels,shuffle=True)
validation_dataset=BertSemanticDataGenerator(validation_inputs,validation_labels,shuffle=False)

In [ ]:
# for d in tqdm(train_dataset):
#     print(d)

In [ ]:
input_layer = tf.keras.layers.Input(shape=(128, 768), name=None)  
flat=tf.keras.layers.Flatten()(input_layer) 
output = tf.keras.layers.Dense(6, activation="softmax")(flat)
model = tf.keras.models.Model(inputs=input_layer, outputs=output)
    
model.summary()

In [ ]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=2
)

In [ ]:
for step,(x_batch_train,labels) in enumerate(tqdm(train_dataset)):
    print(labels.flatten().shape)

In [48]:
label

NameError: name 'label' is not defined

In [49]:
# custom training loop 
import time
## defining a optimizer 
optimizert=tf.keras.optimizers.SGD(learning_rate=1e-3)

## defining loss function 
loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# actually from logits denoting the probability from our custom model layed for each label.It is being fetched before the softmax layer to calculate loss between actual and predicted

batch_size=32
EPOCH=2
train_dataset_size=60000
validation_dataset_size=15000

@tf.function
def train_step(model,x_train,label):
    # Gradiane tape actually records the operation run in forward step
    with tf.GradientTape() as tape:
        #caluculate logits for comparison
        logits_prob=model(x_train,training=True)
        # calculate loss value
        print(label.shape,logits_prob.shape)
        loss_value=loss_fn(label,logits_prob)
    #calculate gradient of trainable variables against the loss
    gradients=tape.gradient(loss_value,model.trainable_weights)
    # update the gradient according to gradient descent
    optimizer.apply_gradients(zip(gradients,model.trainable_weights))
    # update accuracy metric
    train_acc_metric.update_state(lables,logits_prob)
    return loss_value

@tf.function
def validation_step(model,x_validation,label):
    with tf.GradientTape() as tape:
        validation_logit_prob=model(x,training=False)
        val_acc_metric.update_state(label,validation_logit_prob)

def train_model(model,train_dataset,validation_dataset):
    for epoch in range(EPOCHS):
        print('\n Epoch No %d\n' % (epoch,))

        ### training part ###
        for step,(x_batch_train,labels) in enumerate(tqdm(train_dataset)):
            training_loss=train_step(model,x_batch_train,labels)
            
            #log result at every 50 batches
            if step%50==0:
                print("Trainng loss at %d batch of data: %.4f"%(step,float(training_loss)))
        # training accuracy metric at end
        training_accuarcy=train_acc_metric.result()
        print("\nTraining accuracy after %d epoch : %.4f"%(epoch,training_accuracy))
        
        # reset the accuracy metric after every epoch
        train_acc_metric.reset_states()
        
        
        ### validation part ###
        for step,(x_batch_val,labels) in enumerate(tqdm(validation_dataset)):
            validation_step(x_batch_val,labels)
        validation_acc=val_acc_metric.result()
        val_acc_metric.reset_states()
        print("\n validation accuracy : %4.f"%(validation_acc))

train_model(model,train_dataset,validation_dataset)


 Epoch No 0



  0%|          | 0/1875 [00:00<?, ?it/s]

(32, 6) (32, 6)


ValueError: in user code:

    /tmp/ipykernel_34/97174519.py:23 train_step  *
        loss_value=loss_fn(label,logits_prob)
    /opt/conda/lib/python3.7/site-packages/keras/losses.py:141 __call__  **
        losses = call_fn(y_true, y_pred)
    /opt/conda/lib/python3.7/site-packages/keras/losses.py:245 call  **
        return ag_fn(y_true, y_pred, **self._fn_kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:206 wrapper
        return target(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/keras/losses.py:1739 sparse_categorical_crossentropy
        y_true, y_pred, from_logits=from_logits, axis=axis)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:206 wrapper
        return target(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/keras/backend.py:4957 sparse_categorical_crossentropy
        labels=target, logits=output)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:206 

In [ ]:
# from keras.models import load_model
# model=load_model('../input/mymodel/my_model.h5')

In [ ]:
s4 = 'i will killl oyu'
sentence_pairs = np.array([s4])
test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

pro=model.predict(test_data)
print(pro)
print(label_cols[np.argmax(pro)])

In [ ]:
# loading the pre-defined bert model weights
bert_model.Trainable=True

train_dataset=BertSemanticDataGenerator(train_inputs,train_labels,shuffle=True)
validation_dataset=BertSemanticDataGenerator(validation_inputs,validation_labels,shuffle=False)

trained_history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=2
)

In [ ]:
model.trainable_variables

In [ ]:
# submission_df=pd.read_csv("./sample_submission.csv",index_col='id')
# test_df=pd.read_csv("./test.csv")
# label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# print(submission_df.head())
# print(test_df.head())


# test_bert_op=BertSemanticDataGenerator(test_df['comment_text'],None,include_targets=False,shuffle=True)

In [ ]:
# for i,sen in enumerate(tqdm(test_bert_op)):
#     sample_ids = test_df.iloc[i*32:(i+1)*32]['id'] 
#     pred=model.predict(sen)
#     submission_df.loc[sample_ids, label_cols] = pred
#     print(pred)
#     print(sen.shape)

In [ ]:
# submission_df.to_csv('submission.csv')

In [ ]:
# sample_ids = test_df.iloc[0*32:(0+1)*32]['id'] 
# print(sample_ids)
# submission_df.loc[sample_ids]

In [ ]:
model.save("my_model.h5")

In [ ]:
# from keras.preprocessing.sequence import pad_sequences
# # to convert the iput ids array in same size(column no=max(column no))
# tokenizer=transformers.BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
# max_length=128
# bert_model=transformers.TFBertModel.from_pretrained("bert-base-uncased")
# bert_model.trainable=False

# def tokenize(data,tokenizer=tokenizer,max_length=max_length):
# #     input_ids=[]
# #     attention_masks=[]
#     bert_outputs=[]
#     for sentence in tqdm(data):
        
#         encoded_data=tokenizer.batch_encode_plus(
#                         sentence,
#                         add_special_tokens=True,
#                         max_length=max_length,
#                         truncation=True,
#                         return_attention_mask=True,
#                         return_token_type_ids=True,
#                         pad_to_max_length=True,
#                         return_tensors="tf",
#                     )

# #         input_id=np.array(encoded_data["input_ids"],dtype="int32")
# #         attention_mask=np.array(encoded_data["attention_mask"],dtype="int32") 
        
#         bert_output=bert_model(**encoded_data)
#         sequence_output = bert_output.last_hidden_state
#         bert_outputs.append(sequence_output)
#     return bert_outputs
# #         input_ids.append(input_id)
# #         attention_masks.append(attention_mask)
        
# #     return [input_ids,attention_masks]

# bert_op=tokenize(data['comment_text'])
# # input_ids=pad_sequences(bert_op[0],maxlen=max_length,dtype='long',value=0,truncating="post",padding="post")
# # attention_masks=bert_op[1]
# # bert_op.shape

In [ ]:
# len(bert_op)

In [ ]:
# attention_masks=np.array(attention_masks)
# attention_masks=pad_sequences(attention_masks,maxlen=max_length,dtype='long',value=0,truncating="post",padding="post")
# input_ids.shape
# attention_masks.shape

In [ ]:
# # creating batched dataset
# epochs=2
# def create_batch_dataset(data,epochs=epochs,batch_size=batch_size,buffer_size=1000,train=True):
#     dataset=tf.data.Dataset.from_tensor_slices(data)
# #     print(dataset.as_numpy_iterator())
#     if train:
#         dataset=dataset.shuffle(buffer_size=buffer_size)
#         # uses for shuffling the dataset.Select the first buffer_size element from dataset
#     dataset=dataset.repeat(epochs)
#     # just repeat the whole dataset
#     dataset=dataset.batch(batch_size=batch_size)
#     # devide the whole dataset into batch size and create an array of array
#     if train:
#         dataset=dataset.prefetch(1)
#     #     It has no concept of examples vs. batches. examples.prefetch(2) will prefetch two 
#     # elements (2 examples), while examples.batch(20).prefetch(2) will prefetch 2 elements (2 
#     # batches, of 20 examples each).
#     return dataset
# train_dataset=create_batch_dataset((train_inputs,train_masks,train_labels),train=True)
# validation_dataset=create_batch_dataset((validation_inputs,validation_masks,validation_labels),train=True)

In [ ]:
# print(train_dataset.as_numpy_iterator())
# print(validation_dataset.as_numpy_iterator())

In [ ]:
# list(train_dataset.as_numpy_iterator())[0][0].shape

In [ ]:
# for i, (token_ids, masks, labels) in enumerate(tqdm(train_dataset)):
#             print(token_ids.shape)

In [ ]:
# https://www.kaggle.com/nkaenzig/bert-tensorflow-2-huggingface-transformers
# https://www.kaggle.com/satyamkryadav/bert-model-96-77/notebook